### Installing the EconML library

In [1]:
# pip install econml

## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from econml.sklearn_extensions.linear_model import StatsModelsLinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import econml
from econml.orf import DMLOrthoForest
from econml.dml import DML

### Working on dataset and feature engineering

In [3]:
unemployment_df = pd.read_csv('unemployement.csv').apply(lambda x: x.fillna(x.mode().iloc[0]) if x.dtype == 'object' else x.fillna(x.mean()))
minimum_wage_df = pd.read_csv('Mw.csv').apply(lambda x: x.fillna(x.mode().iloc[0]) if x.dtype == 'object' else x.fillna(x.mean()))

minimum_wage_df = minimum_wage_df.apply(
    lambda x: x[np.isfinite(x)] if x.dtype!= 'object' else x
)

unemployment_df = unemployment_df.apply(
    lambda x: x[np.isfinite(x)] if x.dtype!= 'object' else x
)

In [4]:
unemployment_df.columns

Index(['FIPS Code', 'State/Area', 'Year', 'Month',
       'Total Civilian Non-Institutional Population in State/Area',
       'Total Civilian Labor Force in State/Area',
       'Percent (%) of State/Area's Population',
       'Total Employment in State/Area',
       'Percent (%) of Labor Force Employed in State/Area',
       'Total Unemployment in State/Area',
       'Percent (%) of Labor Force Unemployed in State/Area'],
      dtype='object')

In [5]:
unemployment_df.rename(columns={'State/Area': 'State'}, inplace=True)

In [6]:
minimum_wage_df.columns

Index(['Year', 'State', 'State.Minimum.Wage',
       'State.Minimum.Wage.2020.Dollars', 'Federal.Minimum.Wage',
       'Federal.Minimum.Wage.2020.Dollars', 'Effective.Minimum.Wage',
       'Effective.Minimum.Wage.2020.Dollars', 'CPI.Average',
       'Department.Of.Labor.Uncleaned.Data',
       'Department.Of.Labor.Cleaned.Low.Value',
       'Department.Of.Labor.Cleaned.Low.Value.2020.Dollars',
       'Department.Of.Labor.Cleaned.High.Value',
       'Department.Of.Labor.Cleaned.High.Value.2020.Dollars', 'Footnote'],
      dtype='object')

In [7]:
unemployment_df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            29882, 29883, 29884, 29885, 29886, 29887, 29888, 29889, 29890,
            29891],
           dtype='int64', length=29892)

In [8]:
minimum_wage_df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2420, 2421, 2422, 2423, 2424, 2425, 2426, 2427, 2428, 2429],
           dtype='int64', length=2430)

In [9]:
unemployment_df.shape

(29892, 11)

In [10]:
minimum_wage_df.shape

(2430, 15)

In [11]:
# Merge datasets on 'State' and 'Year'
merged_df = pd.merge(minimum_wage_df, unemployment_df, on=['State', 'Year'], how='inner')
merged_df = merged_df.apply(lambda x: x.fillna(x.mode().iloc[0]) if x.dtype == 'object' else x.fillna(x.mode().iloc[0] if x.isnull().any() else x.mean()))

# Feature engineering
merged_df['Minimum_Wage_Change'] = merged_df['State.Minimum.Wage'].pct_change()
merged_df['Minimum_Wage_Treatment'] = np.where(merged_df['Minimum_Wage_Change'] > 0, 1, 0)  # 1 if minimum wage increased, 0 otherwise


merged_df.head()

,Year,State,State.Minimum.Wage,State.Minimum.Wage.2020.Dollars,Federal.Minimum.Wage,Federal.Minimum.Wage.2020.Dollars,Effective.Minimum.Wage,Effective.Minimum.Wage.2020.Dollars,CPI.Average,Department.Of.Labor.Uncleaned.Data,...,Month,Total Civilian Non-Institutional Population in State/Area,Total Civilian Labor Force in State/Area,Percent (%) of State/Area's Population,Total Employment in State/Area,Percent (%) of Labor Force Employed in State/Area,Total Unemployment in State/Area,Percent (%) of Labor Force Unemployed in State/Area,Minimum_Wage_Change,Minimum_Wage_Treatment
0,1976,Alabama,0.0,0.0,2.2,10.0,2.2,10.0,56.9,...,...,1,"2,605,000","1,484,555",57.0,"1,386,023",53.2,"98,532",6.6,NaN,0
1,1976,Alabama,0.0,0.0,2.2,10.0,2.2,10.0,56.9,...,...,2,"2,610,000","1,483,950",56.9,"1,385,675",53.1,"98,275",6.6,NaN,0
2,1976,Alabama,0.0,0.0,2.2,10.0,2.2,10.0,56.9,...,...,3,"2,615,000","1,484,241",56.8,"1,386,793",53.0,"97,448",6.6,NaN,0
3,1976,Alabama,0.0,0.0,2.2,10.0,2.2,10.0,56.9,...,...,4,"2,620,000","1,487,233",56.8,"1,390,787",53.1,"96,446",6.5,NaN,0
4,1976,Alabama,0.0,0.0,2.2,10.0,2.2,10.0,56.9,...,...,5,"2,626,000","1,491,392",56.8,"1,395,320",53.1,"96,072",6.4,NaN,0


In [12]:
merged_df.shape

(27540, 26)

In [13]:
merged_df['Minimum_Wage_Change'] = merged_df['Minimum_Wage_Change'].fillna(merged_df['Minimum_Wage_Change'].mode().iloc[0])

In [14]:
merged_df.isnull().sum()

Year                                                         0
State                                                        0
State.Minimum.Wage                                           0
State.Minimum.Wage.2020.Dollars                              0
Federal.Minimum.Wage                                         0
Federal.Minimum.Wage.2020.Dollars                            0
Effective.Minimum.Wage                                       0
Effective.Minimum.Wage.2020.Dollars                          0
CPI.Average                                                  0
Department.Of.Labor.Uncleaned.Data                           0
Department.Of.Labor.Cleaned.Low.Value                        0
Department.Of.Labor.Cleaned.Low.Value.2020.Dollars           0
Department.Of.Labor.Cleaned.High.Value                       0
Department.Of.Labor.Cleaned.High.Value.2020.Dollars          0
Footnote                                                     0
FIPS Code                                              

In [15]:
merged_df.columns

Index(['Year', 'State', 'State.Minimum.Wage',
       'State.Minimum.Wage.2020.Dollars', 'Federal.Minimum.Wage',
       'Federal.Minimum.Wage.2020.Dollars', 'Effective.Minimum.Wage',
       'Effective.Minimum.Wage.2020.Dollars', 'CPI.Average',
       'Department.Of.Labor.Uncleaned.Data',
       'Department.Of.Labor.Cleaned.Low.Value',
       'Department.Of.Labor.Cleaned.Low.Value.2020.Dollars',
       'Department.Of.Labor.Cleaned.High.Value',
       'Department.Of.Labor.Cleaned.High.Value.2020.Dollars', 'Footnote',
       'FIPS Code', 'Month',
       'Total Civilian Non-Institutional Population in State/Area',
       'Total Civilian Labor Force in State/Area',
       'Percent (%) of State/Area's Population',
       'Total Employment in State/Area',
       'Percent (%) of Labor Force Employed in State/Area',
       'Total Unemployment in State/Area',
       'Percent (%) of Labor Force Unemployed in State/Area',
       'Minimum_Wage_Change', 'Minimum_Wage_Treatment'],
      dtype='object'

In [16]:
# Select relevant features and target
X = merged_df[['Minimum_Wage_Change', 'CPI.Average']]
y = merged_df['Percent (%) of Labor Force Unemployed in State/Area']
T = merged_df['Minimum_Wage_Treatment']  # Treatment variable (e.g., 1 if minimum wage increased, 0 otherwise)

# Split data into training and testing sets
X_train, X_test, y_train, y_test, T_train, T_test = train_test_split(X, y, T, test_size=0.2, random_state=42)

print("y_train shape:", y_train.shape)
print("T_train shape:", T_train.shape)
print("X_train shape:", X_train.shape)


y_train shape: (22032,)
T_train shape: (22032,)
X_train shape: (22032, 2)


In [17]:
X_train = X_train.replace([np.inf, -np.inf], 1e10)  # Replace with a large but finite value
y_train = y_train.replace([np.inf, -np.inf], 1e10)
X_test = X_test.replace([np.inf, -np.inf], 1e10)
y_test = y_test.replace([np.inf, -np.inf], 1e10)
T_train = T_train.replace([np.inf, -np.inf], 0)  # For binary treatment, replace with 0 or 1

print("y_train shape:", y_train.shape)
print("T_train shape:", T_train.shape)
print("X_train shape:", X_train.shape)

print(X_train.isnull().sum())
print(X_test.isnull().sum())
print(y_train.isnull().sum())
print(y_test.isnull().sum())
print(T_train.isnull().sum())
print(T_test.isnull().sum())

y_train shape: (22032,)
T_train shape: (22032,)
X_train shape: (22032, 2)
Minimum_Wage_Change    0
CPI.Average            0
dtype: int64
Minimum_Wage_Change    0
CPI.Average            0
dtype: int64
0
0
0
0


# Econml for Causal Inference

### 1. DoubleML for Average Treatment Effects

In [18]:
double_ml = DML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestClassifier(),
    model_final=StatsModelsLinearRegression(fit_intercept=False),
    discrete_treatment=True
)
double_ml.fit(y_train, T_train, X=X_train, W=None)  
ate_double_ml = double_ml.ate(X = X_test)
cate_double_ml = double_ml.const_marginal_ate(X = X_test)
print("DoubleML ATE:", ate_double_ml)
print("DoubleML CATE:", cate_double_ml)

DoubleML ATE: 2186117426.4330044
DoubleML CATE: [2.18611743e+09]


### 2. OrthoForest for Heterogeneous Treatment Effects

In [19]:
ortho_forest = DMLOrthoForest(
    n_trees=100,                      # Number of trees
    min_leaf_size=10,                 # Minimum samples required to be a leaf node
    max_depth=5,                      # Maximum depth of each tree
    model_T=RandomForestRegressor(),  # Treatment model
    model_Y=RandomForestRegressor(),  # Outcome model
    model_T_final=RandomForestRegressor(),
    model_Y_final=RandomForestRegressor()
)

# Fit the model

ortho_forest.fit(y_train, T_train, X=X_train)
hte_ortho_forest = ortho_forest.const_marginal_ate(X_test)
print("OrthoForest HTE:", hte_ortho_forest)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:  7.3min
[Parall

OrthoForest HTE: -828.7448367343603


[Parallel(n_jobs=-1)]: Done 5508 out of 5508 | elapsed: 20.4min finished
